In [ ]:
from Common import DiGraph
import numpy as np
import pandas as pd
from TopoSortUtils import TopoOrder
from functools import reduce
from random import sample
import random
import time
import sys, os
import math
from DVEsetUtils import DVEset
import itertools
import networkx as nx
import bnlearn as bn
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

from pgmpy.models import BayesianModel,BayesianNetwork
from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator
from pgmpy.sampling import BayesianModelSampling
from pgmpy.inference import VariableElimination

from sklearn.metrics import mean_squared_error 

In [ ]:
G = {'Age':['GoodStudent','SocioEcon', 'RiskAversion', 'DrivingSkill', 'SeniorTrain', 'MedCost'],'SocioEcon':['GoodStudent', 'RiskAversion', 'VehicleYear', 'MakeModel', 'HomeBase', 'AntiTheft', 'OtherCar'],
     'RiskAversion':['VehicleYear', 'MakeModel', 'DrivQuality', 'SeniorTrain', 'HomeBase', 'AntiTheft', 'DrivHist'],'VehicleYear':['RuggedAuto', 'Antilock', 'CarValue', 'Airbag'],'ThisCarDam':['ThisCarCost'],
     'RuggedAuto':['ThisCarDam', 'OtherCarCost', 'Cushioning'],'Accident':['ThisCarDam', 'OtherCarCost', 'MedCost', 'ILiCost'],'MakeModel':['RuggedAuto', 'Antilock', 'CarValue', 'Airbag'],
     'DrivQuality':['Accident'],'Mileage':['Accident', 'CarValue'],'Antilock':['Accident'],'DrivingSkill':['DrivQuality', 'DrivHist'],'SeniorTrain':['DrivingSkill'],'ThisCarCost':['PropCost'],'Theft':['ThisCarCost'],'CarValue':['ThisCarCost', 'Theft']
    ,'HomeBase':['Theft'],'AntiTheft':['Theft'],'OtherCarCost':['PropCost'],'Cushioning':['MedCost'],'Airbag':['Cushioning'],'GoodStudent':[],'OtherCar':[],'DrivHist':[],'MedCost':[],'ILiCost':[],'PropCost':[]}

In [ ]:
V = ['Age','GoodStudent', 'SocioEcon','RiskAversion','DrivingSkill','SeniorTrain','MedCost','VehicleYear','MakeModel','HomeBase','AntiTheft',
 'OtherCar', 'DrivQuality','DrivHist', 'RuggedAuto','Antilock','CarValue','Airbag','ThisCarDam','ThisCarCost','OtherCarCost','Cushioning',
'Accident', 'ILiCost', 'Mileage', 'PropCost', 'Theft']

In [ ]:
def query_report(infer, variables, evidence=None, elimination_order="MinFill", show_progress=False, desc=""):
    if desc:
        print(desc)
    start_time = time.time()
    query_a = infer.query(variables=variables,
                      evidence=evidence,
                      elimination_order=elimination_order,
                      show_progress=show_progress)
    #print(query_a)
    #print(f'--- Query executed in {time.time() - start_time:0,.4f} seconds ---\n')
    return query_a

def get_ordering(infer, variables, evidence=None, elimination_order="MinFill", show_progress=False, desc=""):
    start_time = time.time()
    ordering = infer._get_elimination_order(variables=variables,
                                        evidence=evidence,
                                        elimination_order=elimination_order,
                                        show_progress=show_progress)
    return ordering

In [ ]:
def get_state_probability(self, states):
        # Step 1: Check that all variables and states are in the model.
        #self.check_model()
        for var, state in states.items():
            if var not in self.nodes():
                raise ValueError(f"{var} not in the model.")
            if state not in self.states[var]:
                raise ValueError(f"State: {state} not define for {var}")

        # Step 2: Missing variables in states.
        missing_vars = list(set(self.nodes()) - set(states.keys()))
        missing_var_states = {var: self.states[var] for var in missing_vars}

        # Step 2: Compute the probability
        final_prob = 0
        for state_comb in itertools.product(*missing_var_states.values()):
            temp_states = {
                **{var: state_comb[i] for i, var in enumerate(missing_vars)},
                **states,
            }
            prob = 1
            for cpd in self.cpds:
                index = []
                for var in cpd.variables:
                    index.append(cpd.name_to_no[var][temp_states[var]])
                prob *= cpd.values[tuple(index)]
            final_prob += prob

        return final_prob

In [ ]:
def get_value(phi, Dic):
        index = []
        for var in phi.variables:
            try:
                index.append(phi.name_to_no[var][Dic[var]])
            except KeyError:
                logging.info(f"Using {var} state as number instead of name.")
                index.append(Dic[var])
        return phi.values[tuple(index)]

In [ ]:
model_V = BayesianModel([('Age', 'GoodStudent'), ('Age', 'SocioEcon'), ('Age', 'RiskAversion'), ('Age', 'DrivingSkill'), 
                         ('Age', 'SeniorTrain'), ('Age', 'MedCost'), ('SocioEcon', 'GoodStudent'), ('SocioEcon', 'RiskAversion'), 
                         ('SocioEcon', 'VehicleYear'), ('SocioEcon', 'MakeModel'), ('SocioEcon', 'HomeBase'), ('SocioEcon', 'AntiTheft'), 
                         ('SocioEcon', 'OtherCar'), ('RiskAversion', 'VehicleYear'), ('RiskAversion', 'MakeModel'), ('RiskAversion', 'DrivQuality'), 
                         ('RiskAversion', 'SeniorTrain'), ('RiskAversion', 'HomeBase'), ('RiskAversion', 'AntiTheft'), ('RiskAversion', 'DrivHist'), 
                         ('VehicleYear', 'RuggedAuto'), ('VehicleYear', 'Antilock'), ('VehicleYear', 'CarValue'), ('VehicleYear', 'Airbag'), 
                         ('ThisCarDam', 'ThisCarCost'), ('RuggedAuto', 'ThisCarDam'), ('RuggedAuto', 'OtherCarCost'), ('RuggedAuto', 'Cushioning'), 
                         ('Accident', 'ThisCarDam'), ('Accident', 'OtherCarCost'), ('Accident', 'MedCost'), ('Accident', 'ILiCost'), ('MakeModel', 'RuggedAuto'), 
                         ('MakeModel', 'Antilock'), ('MakeModel', 'CarValue'), ('MakeModel', 'Airbag'), ('DrivQuality', 'Accident'), ('Mileage', 'Accident'), 
                         ('Mileage', 'CarValue'), ('Antilock', 'Accident'), ('DrivingSkill', 'DrivQuality'), ('DrivingSkill', 'DrivHist'), 
                         ('SeniorTrain', 'DrivingSkill'), ('ThisCarCost', 'PropCost'), ('Theft', 'ThisCarCost'), ('CarValue', 'ThisCarCost'), ('CarValue', 'Theft'),
                         ('HomeBase', 'Theft'), ('AntiTheft', 'Theft'), ('OtherCarCost', 'PropCost'), ('Cushioning', 'MedCost'), ('Airbag', 'Cushioning')])


In [ ]:
heuristic = "MinNeighbors"
model = BayesianNetwork.load('insurance.bif', filetype='bif')
inference = BayesianModelSampling(model)#用于采样
infer = VariableElimination(model)

In [ ]:
Bias_I = 0
Bias_II = 0
Bias_III = 0

RMSE_I = 0
RMSE_II = 0
RMSE_III = 0

Time_I = 0
Time_II = 0
Time_III = 0

Final = pd.DataFrame()
In = 0

ct = 0

for i in range(0,1):

    R =  ['OtherCarCost','ThisCarDam','ThisCarCost','PropCost']
    start = time.time()
    M = list(set(V) - set(R)) 
    An_R = DiGraph(G).ancestors(R)
    An_R = DiGraph(G).inducedSubgraph(An_R)
    dve = DVEset(An_R)
    G_sat, G_dve, add_edges = dve.get_set_min_graph(M)
    end = time.time()
    
    edges_T = []
    edges_I = []
    for item in G_dve._graph_dict.items():
        for element in item[1]:
            edges_T.append((item[0],element))
            if (item[0],element) in list(model_V.edges):
                edges_I.append((item[0],element))
                
    model_T = BayesianNetwork()
    model_T = BayesianNetwork(edges_T)
    model_T.add_nodes_from(R)
  
    model_I = BayesianNetwork()
    model_I = BayesianNetwork(edges_I)
    model_I.add_nodes_from(R)
    
    ordering = get_ordering(infer, variables=R, elimination_order=heuristic)
    
    A = query_report(infer, variables=R, elimination_order=ordering)
    
    L = []
    for k in A.assignment([np.argmax(A.values)]):
        L = k
    
    a = np.max(A.values)
    
    for g in [1000, 2500, 5000, 7500, 10000, 12500, 15000, 20000,30000,40000,50000,
             60000,70000,80000,90000,100000]:
        
        Bias_I_I = 0
        Bias_II_I = 0
        Bias_III_I = 0
    
        RMSE_I_I = 0
        RMSE_II_I = 0
        RMSE_III_I = 0
    
        Time_I_I = 0
        Time_II_I = 0
        Time_III_I = 0
    
        for j in range(0,100):
            
            print(ct)
            ct+=1
            df = inference.forward_sample(size=g)
        
            q = time.time()
            model_V.cpds = []
            model_V.fit(data=df,estimator=MaximumLikelihoodEstimator)
            infer_V = VariableElimination(model_V)
            ordering = get_ordering(infer_V, variables=R, elimination_order=heuristic)
            B = query_report(infer_V, variables=R, elimination_order=ordering)
            b = get_value(B,{L[0][0]: L[0][1], L[1][0]: L[1][1], L[2][0]: L[2][1], L[3][0]: L[3][1]})
            p = time.time()
            Time_I_I = Time_I_I + p-q
        
            q = time.time()
            model_T.cpds = []
            model_T.fit(data=df,estimator=MaximumLikelihoodEstimator)
            c = get_state_probability(model_T,{L[0][0]: L[0][1], L[1][0]: L[1][1], L[2][0]: L[2][1], L[3][0]: L[3][1]})
            p = time.time()
            Time_II_I = Time_II_I + p-q
        
            q = time.time()
            model_I.cpds = []
            model_I.fit(data=df,estimator=MaximumLikelihoodEstimator) 
            d = get_state_probability(model_I,{L[0][0]: L[0][1], L[1][0]: L[1][1], L[2][0]: L[2][1], L[3][0]: L[3][1]})
            p = time.time()
            Time_III_I = Time_III_I + p-q  
                  
                    
            Bias_I_I = Bias_I_I + b - a
            Bias_II_I = Bias_II_I + c - a
            Bias_III_I = Bias_III_I + d - a
        
            RMSE_I_I = RMSE_I_I + math.pow((b-a),2)
            RMSE_II_I = RMSE_II_I + math.pow((c-a),2)
            RMSE_III_I = RMSE_III_I + math.pow((d-a),2)
        
        
        Final.loc[In,'Time_I'] = Time_I_I / 100
        Final.loc[In,'Time_II'] = Time_II_I / 100 + end - start
        Final.loc[In,'Time_III'] = Time_III_I / 100  
        
        Final.loc[In,'Bias_I'] = Bias_I_I / 100
        Final.loc[In,'Bias_II'] = Bias_II_I / 100
        Final.loc[In,'Bias_III'] = Bias_III_I / 100 
        
        Final.loc[In,'RMSE_I'] = math.sqrt(RMSE_I_I / 100)
        Final.loc[In,'RMSE_II'] = math.sqrt(RMSE_II_I / 100)
        Final.loc[In,'RMSE_III'] = math.sqrt(RMSE_III_I / 100) 
        
        Final.loc[In,'Sample_size'] = g
        In = In + 1
        

Final

In [ ]:
Final